In [ ]:
# USO de ChromaBD Cohera para crear las funciones el Challenge RAG LLM

Instalar librerias necesarias

```bash
pip install cohere=5.5.8
pip install ipykernel=6.29.4
pip install chromadb=0.5.0
pip install python-docx=1.1.2
pip install tiktoken=0.7.0
pip install langchain=0.2.6


## Impotar librerias

In [1]:
import os
from dotenv import load_dotenv

import chromadb

from docx import Document
import tiktoken
from typing import Union, List

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langdetect import detect
import langid
import cohere

import uuid
import cohere

import time
from ratelimit import limits, sleep_and_retry

from typing import Sequence, Optional, Union,Literal,Any
from chromadb.api.models.Collection import Collection
EmbedRequestTruncate = Union[Literal["NONE", "START", "END"], Any]
EmbeddingType = Union[Literal["float", "int8", "uint8", "binary", "ubinary"], Any]
EmbedInputType = Union[ Literal["search_document", "search_query", "classification", "clustering"], Any]


import numpy as np

In [2]:
# Limites del API de Cohere
EMBED_LIMIT = 5  # llamadas por minuto
RERANK_CHAT_LIMIT = 10  # llamadas por minuto
OTHER_LIMIT = 100  # llamadas por minuto

# @sleep_and_retry
# @limits(calls=EMBED_LIMIT, period=60)

In [3]:
load_dotenv()
COHERE_api_key: str = os.getenv("COHERE_API_KEY")

co = cohere.Client(COHERE_api_key)

## Crear una instancia de la clase chromadb con Persistencia

In [4]:
# client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")

## Crear un collection

In [5]:
# client.get_or_create_collection(name="respuestas_api")

## Listar los elementos collections

In [7]:
# client.list_collections()

## Eliminar los elementos collections

In [10]:
# client.delete_collection(name="document_chunks")
# client.delete_collection(name="respuestas_api")
# client.delete_collection(name="document_chunks2")

TEST API

In [10]:
import requests

# url = "http://localhost:8003/api/ask"
url = "http://localhost:8003/api/ask_cohere"
payload = {
    "question": "quien es Zara?",
    "user_name": "Mvx"
}
headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())


{'answer': 'Zara es un intrépido explorador que descubre un antiguo artefacto que podría ser clave para la paz en la galaxia Zenthoria. 🚀🛸🌌️ 😊', 'status': 200}


In [4]:
import requests

url = "http://localhost:8003/api/"
payload = {
    "question": "quien es Zara?",
    "user_name": "Mvx"
}
headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())


{'answer': 'Hola Pi Consulting'}


## Funciones

In [13]:
def print_collections(collections:list) -> None:
    """
    Descripción:
    ------------
    Prints the collections in the database de forma legible para las ultimas versiones de Chromadb.
    En las ultimas versiones no se ve el objeto collection, por lo que se debe de cambiar el print
    
    Parameters:
    -----------
        - collections: list
            Es una lista de objetos de tipo collection
    
    Returns:
    --------
        - None
            imprime en pantalla el nombre de las colecciones
            
    
    Example:
    --------
    >>> import chromadb
    >>> client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")
    >>> client.create_collection(name="respuestas_api")
    >>> print_collections(collections)
    """
    for collection in collections:
        print(f"Collection(name={collection.name})")

def print_verbose(msg: str,verbose:bool) -> None:
    '''
    Description:
    ------------
    Esta función imprime un mensaje en pantalla si verbose es True
    
    Parameters:
    -----------
        - msg: str
            Es el mensaje que se desea imprimir en pantalla
        - verbose: bool
            Es un booleano que indica si se imprime o no el mensaje en pantalla
            
    Returns:
    --------
        - None
            No retorna ningún valor solo imprime en pantalla el mensaje
            
    Example:
    --------
    >>> print_verbose("Hola Mundo", True)
    '''
    print(msg) if verbose else None



def num_tokens_from_string(string: str, encoding_name: str="cl100k_base") -> int:
    """
    Description:
    ------------
        Esta función recibe un string y devuelve el número de tokens que contiene.

    Parameters:
    -----------
        - string: str
            Es el string que se desea tokenizar.
        - encoding_name: str
            Es el nombre del encoding que se desea utilizar para tokenizar el string.
    
    Returns:
    --------
        - num_tokens:int
            Retorna el número de tokens que contiene el string.
    """
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def doc_to_paragraphs(file:str='F:\Proyectos\PiConsulting\challenge_rag_llm\docs\documento.docx'):
    '''
    Description:
    ------------
        Esta función recibe un archivo .docx y devuelve una lista con los párrafos del documento
        y una lista con el número de tokens de cada párrafo.

    Parameters:
    -----------
        - file: str
            Es la ruta del archivo .docx que se desea tokenizar.
    
    Returns:
    --------
        - ntoken_list: list
            Retorna una lista con el número de tokens de cada párrafo.
        - list_parrafo: list
            Retorna una lista con los párrafos del documento.
    
    Example:
    --------
    >>> ntoken_list, list_parrafo = doc_to_paragraphs(file)

    '''
    # Abre el archivo .docx
    doc = Document(file)

    list_parrafo = [parrafo.text for parrafo in doc.paragraphs if (parrafo.text != "")and(parrafo.text != " ")]
    ntoken_list = [num_tokens_from_string(parrafo) for parrafo in list_parrafo]

    return ntoken_list, list_parrafo



    
def split_text(
        input: Union[str,
        List[str]],
        unir_list:bool=False, 
        chunk_size: int=300,
        chunk_overlap: int=0,
        separators:list=["\n\n", "\n",],
        verbose=False
)->list:
    '''
    Description:
    ------------
        Esta función recibe un texto o lista  y lo divide en chunks de un tamaño especificado.
        Por defecto si es una lista no se unirá, si se desea unir se debe de especificar en el
        parámetro unir_list=True
        Nota: si se especifica unir_list=False  y se pasa separators ["\n\n", "\n"] los chunks
        se dividirán por los saltos de linea para cada elemento de la lista de strings.
        Si se especifica unir_list=False y separators = ["\n\n", "\n"] los chunks no tomaran en cuanta chunk_size y chunk_overlap

    Parameters:
    -----------
        - input: str or list
            Es el texto que se desea dividir en chunks.
        - chunk_size: int
            Es el tamaño de cada chunk.
        - chunk_overlap: int
            Es el solapamiento entre chunks.
        - separators: list
            Es una lista con los separadores que se desean utilizar para dividir el texto.
            ejemplo: ["\n\n", "\n", ",", ";", "."]
        - verbose: bool
            Es un booleano que indica si se imprime o no el mensaje en pantalla
        - unir_list: bool   
            Es un booleano que indica si se desea unir la lista de strings en un solo string.

    
    Returns:
    --------
        - chunks: list
            Retorna una lista con los chunks del texto.
    
    Example:
    --------
    >>> chunks = split_text(text)

    >>> docs = split_text(input=list_parrafo,chunk_size=500, chunk_overlap=20,
                            unir_list=False,separators=None,verbose=False)

    >>> docs = split_text(input=list_parrafo, chunk_size=500, chunk_overlap=20,
                            unir_list=False,separators=["\n\n", "\n"],verbose=False)
    
    '''
    if isinstance(input, str):
        input = [input]
    elif isinstance(input, list):
        if unir_list:
            input = [" ".join(input)]
    print_verbose(input,verbose)

    # Configuración del splitter
    text_splitter = RecursiveCharacterTextSplitter(
        separators=separators,          # Separadores a utilizar
        chunk_size=chunk_size,          # Tamaño de cada chunk
        chunk_overlap=chunk_overlap,    # Solapamiento entre chunks
    )

    # Crear documentos divididos
    docs = text_splitter.create_documents(input)

    # Mostrar resultado
    for doc in docs:
        print_verbose(msg=f'Chunk: {doc.page_content}', verbose=verbose)

    return docs


def get_context(collection:Collection,query:str,n_results:int=1):
    '''
    Description:
    ------------
    Esta función recibe un query y devuelve el contexto de la respuesta.
    
    Parameters:
    -----------
        - collection: Collection
            Es la colección con la que se desea realizar la consulta.
        - query: str
            Es la pregunta que se desea realizar.
        - n_results: int
            Es el número de resultados que se desean obtener.
            por defecto es 1.
    
    Returns:
    --------
        - results: dict
            Retorna un diccionario con los resultados de la consulta.
        - contexto: str
            Retorna el contexto de la respuesta.
    
    Example:
    --------
    >>>query = "Quien es Zara?"
    >>> results,contexto = get_context(query)
    '''

    # query_embedding = co.embed(texts=[query]).embeddings[0]
    query_embedding = co.embed(texts=[query],model='embed-multilingual-v3.0',input_type ='search_query').embeddings[0] # search_query" or "search_document"
    results = collection.query(query_embeddings=query_embedding, n_results=n_results,include=['documents']) #include=['embeddings','documents']
    contexto = ' '.join([result for result in results['documents'][0]])
    # contexto  = results['documents'][0][0]
    contexto
    return results,contexto

# def get_context(collection:Collection,query:str,n_results:int=1):
#     # query_embedding = co.embed(texts=[query]).embeddings[0]
#     query_embedding = co.embed(texts=[query],model='embed-multilingual-v3.0',input_type ='search_query').embeddings[0] # search_query" or "search_document"
#     results = collection.query(query_embeddings=query_embedding, n_results=n_results,include=['documents']) #include=['embeddings','documents']
#     contexto = ' '.join([result for result in results['documents'][0]])
#     # contexto  = results['documents'][0][0]
#     contexto
#     return results,contexto


def add_documents_to_collection(
        collection:Collection,
        docs:list,
        model:str ='embed-multilingual-v3.0',
        input_type: Optional[EmbedInputType]='search_query',
        embedding_types:Optional[EmbedInputType]= None,
        metadata_options:dict = dict(),

    )->Collection:
    '''
    Description:
    ------------
    Esta función recibe una colección y una lista de documentos y los agrega a la colección.

    Parameters:
    -----------
        - collection: Collection
            Es la colección a la que se desea agregar los documentos.
        - docs: list
            Lista de documentos que se desean agregar a la colección.
        - model: str
            Es el modelo que se desea utilizar para realizar el embedding.
            Por defecto es 'embed-multilingual-v3.0'.
            Los modelos disponibles son:(verificar en la documentación de Cohere)
                - embed-english-v3.0 1024
                - embed-multilingual-v3.0 1024
                - embed-english-light-v3.0 384
                - embed-multilingual-light-v3.0 384
                - embed-english-v2.0 4096
                - embed-english-light-v2.0 1024
                - embed-multilingual-v2.0 768
        - input_type: Optional[EmbedInputType]
            Es el tipo de input que se desea utilizar.
            Por defecto es 'search_query'.
            Los tipos de input disponibles son:(verificar en la documentación de Cohere)
                - search_document
                - search_query
                - classification
                - clustering
        - embedding_types: Optional[EmbedRequestTruncate]
            Es el tipo de embedding que se desea utilizar.
            Por defecto es "float".
            Los tipos de embedding disponibles son:(verificar en la documentación de Cohere)
                - "float"
                - "int8"
                - "uint8"
                - "binary"
                - "ubinary"
        - metadata_options: dict
            Es un diccionario con los metadatos que se desean agregar a los documentos.
            Por defecto es un diccionario vacío.
    
    Returns:
    --------
        - collection: Collection
            Retorna la colección con los documentos agregados.

    Example:
    --------
    >>> import chromadb
    >>> client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")
    >>> collection = client.get_or_create_collection(name="respuestas_api")
    >>> docs = split_text(['text1','text2'])
    >>> collection = add_documents_to_collection(collection,docs)


    '''
    for doc in docs:
        doc = doc.page_content
        uuid_name = uuid.uuid1()
        embedding = co.embed(texts=[doc], model=model, input_type=input_type,
            embedding_types=embedding_types).embeddings[0] # search_query" or "search_document"
        print("document for", uuid_name)
        # doc.page_content
        
        collection.add(ids=[str(uuid_name)],
                    embeddings=embedding,
                    metadatas=metadata_options,
                    documents=doc)
    return collection

@sleep_and_retry
@limits(calls=EMBED_LIMIT, period=60)
def detect_language(query:str):
    '''
    Description:
    ------------
        Esta función recibe un query y devuelve el idioma del query.
        Si el idioma no es español, inglés o portugués, se solicita al LLM que interprete el idioma.

    Parameters:
    -----------
        - query: str
            es el texto que se desea detectar el idioma.
    
    Returns:
    --------
        - language: str
        retorno el idioma del texto en formato de dos letras.

    Example:
    --------
    >>> query = "Quien es Zara?"
    >>> language = detect_language(query)
    es
    >>>
    '''
    
    language = langid.classify(query)[0]
    if not language in ['es','en','pt']:
        response = co.chat(
        chat_history=[
            {"role": "SYSTEM", "message": f"""Eres  un detector muy preciso de idiomas,dime el idioma en una sola palabra (es, en, pt) de la siguiente pregunta : """},
        ],
        message=f"""{query}""",
        # connectors=[{"id": "translation"}],
        seed=44,
        temperature=0,
        )
        language = response.text
    return language

@sleep_and_retry
@limits(calls=EMBED_LIMIT, period=60)
def generate_response(query:str, contexto:str):
    '''
    Description:
    ------------
        Esta función recibe un query y un contexto y devuelve una respuesta en el mismo idioma que el query.
    
    Parameters:
    -----------
        - query: str
            Es la pregunta que se desea realizar.
        - contexto: str
            Es el contexto de la respuesta.

    Returns:
    --------
        - respuesta: str
            Retorna la respuesta en el mismo idioma que el query.

    Example:
    --------
    >>> query = "Quien es Zara?"
    >>> contexto = "Zara es una empresa de moda"
    >>> respuesta = generate_response(query, contexto)
    >>>
    '''
    language = detect_language(query)

    dict_important1 =  {
            "es":"Debes ser lo más conciso y preciso posible en la entrega de información (20 palabras o menos) y traducir tu respuesta si es necesario a partir del siguiente contexto:",
            "en":"You must be as concise and precise as possible in delivering information (20 words or less) and translate your response if necessary from the following context:",
            "pt":"Você deve ser o mais conciso e preciso possível na entrega de informações (20 palavras ou menos) e traduzir sua resposta, se necessário, a partir do seguinte contexto:",
            }

    dict_important2 = {
            "es": "Responde en una sola oración, en el mismo idioma que la pregunta, incluyendo emojis que resuman el contenido de la respuesta, y siempre en tercera persona.",
            "en": "Answer in one sentence, in the same language as the question, including emojis that summarize the content of the answer, and always in third person.",
            "pt": "Responda em uma frase, no mesmo idioma da pergunta, incluindo emojis que resumam o conteúdo da resposta, e sempre na terceira pessoa.",
        }
    dict_important3  = {
            "es": "Responde en español, si el texto esta en español traduce la respuesta al español.",
            "en": "Answer in English, if the text is in español, translate the answer to English.",
            "pt": "Responda em português, se o texto estiver em español, traduza a resposta para o português.",
        }

    response = co.chat(
        chat_history=[
            {"role": "SYSTEM", "message": f"""{dict_important1[language]}
            {contexto}
            NOTA IMPORTANTE:
            {dict_important2[language]}

            """},
            
            # {
            #     "role": "CHATBOT",
            #     "message": f"ya identifique el idioma del siguiente mensaje :{query} procedere a responder en el mismo idioma.",
            # },
        ],
        message=f"""{query} {dict_important3[language]}""",
        #"realizar búsqueda web antes de responder a la pregunta. También puedes usar tu propio conector personalizado."
        # · contenedores de traducion y busqueda en contexto
        # connectors=[{"id": "translation"}],
        seed=44,
        temperature=0,
        # conversation_id='user_defined_id_1',
        # model = 'command-r-plus',
        # model = 'command-r',
    )
    respuesta =response.text

    return respuesta




In [14]:
# collections = client.list_collections()
# print_collections(collections)

In [15]:
ntoken_list, list_parrafo = doc_to_paragraphs('F:\Proyectos\PiConsulting\challenge_rag_llm\docs\documento.docx')

In [16]:
docs = split_text(input=list_parrafo, chunk_size=500, chunk_overlap=20,unir_list=False,separators=None,verbose=False)#separators=["\n\n", "\n"]
docs

[Document(page_content='Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.'),
 Document(page_content='Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.'),
 Document(page_content='Naturaleza Deslumbrante: En lo profundo de la selva ama

In [17]:
# client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")

In [18]:
# collection_name = 'document_chunks'
# # collection = client.get_or_create_collection(name=collection_name)
# collection = client.get_or_create_collection(name=collection_name)
# metadata_options = {
#     "hnsw:space": "cosine"  # You can change this to "ip" or "cosine" if needed
# }

In [19]:
# # import chromadb
# # client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")
# # collection = client.get_or_create_collection(name="document_chunks")
# # docs = split_text(['text1','text2'])
# collection = add_documents_to_collection(collection,docs,metadata_options=metadata_options,embedding_types=None)

In [20]:
client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")
collection = client.get_or_create_collection(name="document_chunks")
model = 'embed-multilingual-v3.0'
input_type = 'search_query'
embedding_types = None
client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")
collection = client.get_or_create_collection(name="document_chunks")
model = 'embed-multilingual-v3.0'
input_type = 'search_query'
embedding_types = None
metadata_options = {
    "hnsw:space": "cosine"  # You can change this to "ip" or "cosine" if needed
}
collection = add_documents_to_collection(
    collection,
    docs,
    model= model,
    input_type=input_type,
    metadata_options=metadata_options,
    embedding_types=embedding_types)
# collection = add_documents_to_collection(collection,docs,model,input_type)

document for a2005c0d-35af-11ef-9f0c-4c77cb1e1a4e
document for a2411a38-35af-11ef-a65c-4c77cb1e1a4e
document for a2680c41-35af-11ef-aaf2-4c77cb1e1a4e
document for a28dbd70-35af-11ef-8422-4c77cb1e1a4e
document for a2c7c1cf-35af-11ef-8b20-4c77cb1e1a4e


In [21]:
# from typing import Sequence, Optional, Union,Literal,Any
# from chromadb.api.models.Collection import Collection
# EmbedRequestTruncate = Union[Literal["NONE", "START", "END"], Any]
# EmbeddingType = Union[Literal["float", "int8", "uint8", "binary", "ubinary"], Any]
# EmbedInputType = Union[
#     Literal["search_document", "search_query", "classification", "clustering"], Any
# ]


# import numpy as np
# def add_documents_to_collection(
#         collection:Collection,
#         docs:list,
#         model:str ='embed-multilingual-v3.0',
#         input_type: Optional[EmbedInputType]='search_query',
#         embedding_types:Optional[EmbedRequestTruncate]= "float",
#         metadata_options:dict = metadata_options

#     ):
#     for doc in docs:
#         doc = doc.page_content
#         uuid_name = uuid.uuid1()
#         embedding = co.embed(texts=[doc], model=model, input_type=input_type,
#             embedding_types=embedding_types).embeddings[0] # search_query" or "search_document"
#         print("document for", uuid_name)
#         # doc.page_content
        
#         collection.add(ids=[str(uuid_name)],
#                     embeddings=embedding,
#                     metadatas=metadata_options,
#                     documents=doc)
#     return collection

In [22]:
# embedding = co.embed(texts=list_parrafo,model='embed-multilingual-v3.0',input_type ='search_query').embeddings[0] # search_query" or "search_document"

In [23]:
# list_parrafo

In [24]:
# import numpy as np
# for doc in docs:
#     doc = doc.page_content
#     uuid_name = uuid.uuid1()
#     embedding = co.embed(texts=[doc],model='embed-multilingual-v3.0',input_type ='search_query').embeddings[0] # search_query" or "search_document"
#     print("document for", uuid_name)
#     # doc.page_content
    
#     collection.add(ids=[str(uuid_name)],
#                 embeddings=embedding,
#                 metadatas=metadata_options,
#                 documents=doc)

In [28]:
# query ='quien es Zara?'
# query ='Who is Zara?'
# query ='Quem é Zara?'
# query ='What did Emma decide to do?'
# query ='¿Qué decidió hacer Emma?'
# query ='O que Emma decidiu fazer?'
query ='What is the name of the magical flower?'
# query ='¿Cuál es el nombre de la flor mágica?'
# query ='Qual é o nome da flor mágica?'
# query ='de que trata el texto?'

results,contexto = get_context(collection,query,5)
respuesta = generate_response(query,contexto)
print(respuesta)

The magical flower is called the "Moonlight" 🌺.


In [29]:

test_language_data = {'quien es Zara?':'es',
                        'Who is Zara?':'en',
                        'Quem é Zara?':'pt',
                        'What did Emma decide to do?':'en',
                        '¿Qué decidió hacer Emma?':'es',
                        'O que Emma decidiu fazer?':'pt',
                        'What is the name of the magical flower?':'en',
                        '¿Cuál es el nombre de la flor mágica?':'es',
                        'Qual é o nome da flor mágica?':'pt'}

def test_detect_language(test_language_data):
    '''
    Función que realiza pruebas de detección de idioma
    Description: 
    ------------
    Esta función recibe un diccionario con preguntas y su respectivo idioma y devuelve un mensaje de éxito si las pruebas pasan.
    la variable test_language_data es un diccionario con preguntas y su respectivo idioma se encuentra en el config_test.yaml.   
 
    '''
    for query,language in test_language_data.items():
        assert detect_language(query) == language # Que hace assert: 
    return 'Test Detectar lenguaje: [PASS]'

test_detect_language(test_language_data)





'Pruebas pasadas con éxito'

In [31]:
print('Test Detectar lenguaje: [PASS]')

Test Detectar lenguaje: [PASS]


In [ ]:

try:

except Exception as e:
            logger.error(f"Error creating retrieval chain: {e}")
            return jsonify({"error": str(e)}), 500

# BORRADOR

instalar librerias

In [38]:
# import chromadb.utils.embedding_functions as embedding_functions
# cohere_ef  = embedding_functions.CohereEmbeddingFunction(api_key=COHERE_api_key,  model_name="large")
# cohere_ef(texts=["document1","document2"])






In [39]:
# cohere_ef.__call__(["document1","document2"])


In [40]:
# cohere_ef  = embedding_functions.CohereEmbeddingFunction(
#         api_key=COHERE_api_key, 
#         model_name="multilingual-22-12")

# multilingual_texts  = [ 'Hello from Cohere!', 'مرحبًا من كوهير!',
#         'Hallo von Cohere!', 'Bonjour de Cohere!',
#         '¡Hola desde Cohere!', 'Olá do Cohere!',
#         'Ciao da Cohere!', '您好，来自 Cohere！',
#         'कोहिअर से नमस्ते!'  ]

# cohere_ef(multilingual_texts)

In [ ]:
# pip install cohere



In [ ]:
# import chromadb
# chroma_client = chromadb.Client()
# # This allows us to create a client that connects to the server
# collection = chroma_client.create_collection(name="my_collection_test")


In [19]:
import os
from dotenv import load_dotenv
load_dotenv()



True

In [2]:
COHERE_api_key: str = os.getenv("COHERE_API_KEY")

In [3]:
import chromadb
client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")

In [4]:
client.create_collection(name="my_collection_test")

Collection(name=my_collection_test)

In [230]:
client.list_collections()

[Collection(name=document_chunks)]

In [4]:
def print_collections(collections):
    for collection in collections:
        print(f"Collection(name={collection.name})")

collections = client.list_collections()
print_collections(collections)

Collection(name=document_chunks)


eliminar los elementos collections

In [ ]:
client.delete_collection(name="my_collection_test")
client.delete_collection(name="my_collection")

In [8]:
collections = client.list_collections()
print_collections(collections)

NameError: name 'print_collections' is not defined

In [7]:
client.list_collections()

[]

In [5]:
def num_tokens_from_string(string: str, encoding_name: str="cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [18]:
from docx import Document
import tiktoken
def doc_to_chromadb_paragraphs(file:str='F:\Proyectos\PiConsulting\challenge_rag_llm\docs\documento.docx'):
    '''
    Description:
    ------------
    Esta función recibe un archivo .docx y lo convierte a un objeto de ChromaDB
    '''
    # Abre el archivo .docx
    doc = Document(file)
    ntoken_list = []
    list_parrafo = []
    # Lee el contenido del documento
    for paragraph in doc.paragraphs:
        if paragraph.text == "":
            continue
        parrafo = paragraph.text
        list_parrafo.append(parrafo)
        print(parrafo)
        ntoken_list.append(num_tokens_from_string(parrafo))
        print('')
    
    ls_parrafo =[paragraph.text for paragraph in doc.paragraphs]
    ls_parrafo = [parrafo for parrafo in ls_parrafo if parrafo != ""]

    ntoken_list = [tok  for tok in ntoken_list if tok > 0 ]
    return ntoken_list, list_parrafo

ntoken_list, list_parrafo = doc_to_chromadb_paragraphs()

Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.

Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.

Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como "Luz de Luna" florece solo durante la 

In [ ]:
%%timeit
ntoken_list = []
list_parrafo = []
for paragraph in doc.paragraphs:
    if paragraph.text == "":
        continue
    parrafo = paragraph.text
    list_parrafo.append(parrafo)
    # print(parrafo)
    ntoken_list.append(num_tokens_from_string(parrafo))
    # print('')


In [ ]:
%%timeit
    
ntoken_list = [num_tokens_from_string(parrafo) for parrafo in list_parrafo]

ntoken_list = [tok  for tok in ntoken_list if tok > 0 ]

In [ ]:
# de que trata el texto?

In [9]:
# file:str='F:\Proyectos\PiConsulting\challenge_rag_llm\docs\documento.docx'
# # Abre el archivo .docx
# doc = Document(file)
# ntoken_list = []
# list_parrafo = []
# # Lee el contenido del documento
# for paragraph in doc.paragraphs:
#     parrafo = paragraph.text
#     list_parrafo.append(parrafo)
#     print(parrafo)
#     ntoken_list.append(num_tokens_from_string(parrafo))
#     print('')

# ls_parrafo =[paragraph.text for paragraph in doc.paragraphs]

# ntoken_list = [tok  for tok in ntoken_list if tok > 0 ]


Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.

Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.

Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como "Luz de Luna" florece solo durante la 

In [155]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    # chunk_size=100,
    # chunk_overlap=20,
    # length_function=len,
    # is_separator_regex=False,
    separators=["\n\n", "\n"], chunk_size=20, chunk_overlap=0
)

docs = text_splitter.create_documents(list_parrafo)


In [157]:
list_parrafo

['Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.',
 'Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.',
 'Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como "Luz de Luna" florece solo dur

In [185]:
input = [
    "Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.",
    "Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.",
    "Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como 'Luz de Luna' florece solo durante la noche. Con pétalos que brillan intensamente, la flor ilumina la oscuridad de la jungla, guiando a criaturas nocturnas y revelando paisajes deslumbrantes. Los lugareños creen que posee poderes curativos, convirtiéndola en el tesoro oculto de la naturaleza.",
    "Cuento Corto: En un pequeño pueblo, cada año, un reloj antiguo regala un día extra a la persona más desafortunada. Emma, una joven huérfana, es la elegida este año. Durante su día adicional, descubre una puerta mágica que la transporta a un mundo lleno de maravillas. Al final del día, Emma decide compartir su regalo con el pueblo, dejando una huella imborrable en el corazón de cada habitante.",
    "Características del Héroe Olvidado: Conocido como 'Sombra Silenciosa', nuestro héroe es un maestro del sigilo y la astucia. Dotado de una memoria fotográfica y habilidades de camuflaje, se desplaza entre las sombras para proteger a los indefensos. Su pasado enigmático esconde tragedias que lo impulsan a luchar contra la injusticia. Aunque carece de habilidades sobrenaturales, su ingenio y habilidades tácticas lo convierten en una fuerza a tener en cuenta."
]
print("\n".join(input))

Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.
Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.
Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como 'Luz de Luna' florece solo durante la no

In [210]:
def print_verbose(msg: str,verbose:bool) -> None:
    '''
    Description:
    ------------
    Esta función imprime un mensaje en pantalla si verbose es True
    
    Parameters:
    -----------
        - msg: str
            Es el mensaje que se desea imprimir en pantalla
        - verbose: bool
            Es un booleano que indica si se imprime o no el mensaje en pantalla
            
    Returns:
    --------
        - None
            No retorna ningún valor solo imprime en pantalla el mensaje
            
    Example:
    --------
    >>> print_verbose("Hola Mundo", True)
    '''
    print(msg) if verbose else None

    

In [228]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Lista de textos/párrafos a dividir
list_parrafo = [
    """Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas,los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.""",
    "Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.",
    "Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como 'Luz de Luna' florece solo durante la noche. Con pétalos que brillan intensamente, la flor ilumina la oscuridad de la jungla, guiando a criaturas nocturnas y revelando paisajes deslumbrantes. Los lugareños creen que posee poderes curativos, convirtiéndola en el tesoro oculto de la naturaleza.",
    "Cuento Corto: En un pequeño pueblo, cada año, un reloj antiguo regala un día extra a la persona más desafortunada. Emma, una joven huérfana, es la elegida este año. Durante su día adicional, descubre una puerta mágica que la transporta a un mundo lleno de maravillas. Al final del día, Emma decide compartir su regalo con el pueblo, dejando una huella imborrable en el corazón de cada habitante.",
    "Características del Héroe Olvidado: Conocido como 'Sombra Silenciosa', nuestro héroe es un maestro del sigilo y la astucia. Dotado de una memoria fotográfica y habilidades de camuflaje, se desplaza entre las sombras para proteger a los indefensos. Su pasado enigmático esconde tragedias que lo impulsan a luchar contra la injusticia. Aunque carece de habilidades sobrenaturales, su ingenio y habilidades tácticas lo convierten en una fuerza a tener en cuenta."
]
# list_parrafo = ['''
#     Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.\n
#     Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.\n
#     Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como 'Luz de Luna' florece solo durante la noche. Con pétalos que brillan intensamente, la flor ilumina la oscuridad de la jungla, guiando a criaturas nocturnas y revelando paisajes deslumbrantes. Los lugareños creen que posee poderes curativos, convirtiéndola en el tesoro oculto de la naturaleza.\n
#     Cuento Corto: En un pequeño pueblo, cada año, un reloj antiguo regala un día extra a la persona más desafortunada. Emma, una joven huérfana, es la elegida este año. Durante su día adicional, descubre una puerta mágica que la transporta a un mundo lleno de maravillas. Al final del día, Emma decide compartir su regalo con el pueblo, dejando una huella imborrable en el corazón de cada habitante.\n
#     Características del Héroe Olvidado: Conocido como 'Sombra Silenciosa', nuestro héroe es un maestro del sigilo y la astucia. Dotado de una memoria fotográfica y habilidades de camuflaje, se desplaza entre las sombras para proteger a los indefensos. Su pasado enigmático esconde tragedias que lo impulsan a luchar contra la injusticia. Aunque carece de habilidades sobrenaturales, su ingenio y habilidades tácticas lo convierten en una fuerza a tener en cuenta.'''
# ]
# importar libreria para definir el tipo de parametro

from typing import Union, List
def split_text(
        input: Union[str,
        List[str]],
        unir_list:bool=False, 
        chunk_size: int=300,
        chunk_overlap: int=0,
        separators:list=["\n\n", "\n",],
        verbose=False
)->list:
    '''
    Description:
    ------------
        Esta función recibe un texto o lista  y lo divide en chunks de un tamaño especificado.
        Por defecto si es una lista no se unirá, si se desea unir se debe de especificar en el
        parametro unir_list=True
        Nota: si se especifica unir_list=False  y se pasa separators ["\n\n", "\n"] los chunks
        se dividirán por los saltos de linea para cada elemento de la lista de strings.
        Si se especifica unir_list=False y separators = ["\n\n", "\n"] los chunks no tomaran en cuanta chunk_size y chunk_overlap

    Parameters:
    -----------
        - input: str or list
            Es el texto que se desea dividir en chunks.
        - chunk_size: int
            Es el tamaño de cada chunk.
        - chunk_overlap: int
            Es el solapamiento entre chunks.
        - separators: list
            Es una lista con los separadores que se desean utilizar para dividir el texto.
            ejemplo: ["\n\n", "\n", ",", ";", "."]
        - verbose: bool
            Es un booleano que indica si se imprime o no el mensaje en pantalla
        - unir_list: bool   
            Es un booleano que indica si se desea unir la lista de strings en un solo string.

    
    Returns:
    --------
        - chunks: list
            Retorna una lista con los chunks del texto.
    
    Example:
    --------
    >>> chunks = split_text(text)

    >>> docs = split_text(input=list_parrafo,chunk_size=500, chunk_overlap=20,
                            unir_list=False,separators=None,verbose=False)

    >>> docs = split_text(input=list_parrafo, chunk_size=500, chunk_overlap=20,
                            unir_list=False,separators=["\n\n", "\n"],verbose=False)
    '''
    if isinstance(input, str):
        input = [input]
    elif isinstance(input, list):
        if unir_list:
            input = [" ".join(input)]
    print_verbose(input,verbose)

    # Configuración del splitter
    text_splitter = RecursiveCharacterTextSplitter(
        separators=separators,          # Separadores a utilizar
        chunk_size=chunk_size,          # Tamaño de cada chunk
        chunk_overlap=chunk_overlap,    # Solapamiento entre chunks
    )

    # Crear documentos divididos
    docs = text_splitter.create_documents(input)

    # Mostrar resultado
    for doc in docs:
        print_verbose(msg=f'Chunk: {doc.page_content}', verbose=verbose)

    return docs



[Document(page_content='Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas,los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos. Ficción Tecnológica: En un futuro distópico, la inteligencia'),
 Document(page_content="la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia. Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una 

In [15]:
docs[0].page_content

'Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.'

In [9]:
text_splitter.split_text(docs[0].page_content)

['Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.']

In [38]:
docs

[Document(page_content='Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.')]

In [10]:
import uuid
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_spliter_chunck(text:str='No Ingresaste texto', max_token:int = 100,chunk_overlap=30):
    '''
    Description:
    ------------
    Esta función recibe un texto y lo divide en partes de max_token
    '''

    text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n"], chunk_size=max_token, chunk_overlap=chunk_overlap)

    docs = text_splitter.create_documents([text])

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n"], chunk_size=20, chunk_overlap=10,keep_separator=True,is_separator_regex=True)

In [32]:
# docs = text_splitter.create_documents([list_parrafo[0]])
docs = text_splitter.create_documents(list_parrafo)

In [16]:
docs

[Document(page_content='Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.'),
 Document(page_content='Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.'),
 Document(page_content='Naturaleza Deslumbrante: En lo profundo de la selva ama

In [17]:
# from chromadb.utils import embedding_functions
# COHERE_api_key: str = os.getenv("COHERE_API_KEY")
# #openai_ef = embedding_functions.OpenAIEmbeddingFunction(model_name="text-embedding-ada-002")
# cohere_ef  = embedding_functions.CohereEmbeddingFunction(api_key=COHERE_api_key,  model_name="large")
# metadata_options = {
#     "hnsw:space": "cosine"  # You can change this to "ip" or "cosine" if needed
# }


# collection = client.get_or_create_collection(
#     name="my_collection", metadata=metadata_options, embedding_function=cohere_ef)

In [31]:
# import chromadb.utils.embedding_functions as embedding_functions
# cohere_ef  = embedding_functions.CohereEmbeddingFunction(api_key=COHERE_api_key,  model_name="large")
# # cohere_ef.__call__(input=["document1","document2"])
# # 

In [28]:
# docs[0].page_content

'Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.'

In [35]:
# cohere_ef.__call__(input=[tuple('document1')])
# # cohere_ef.__call__(input=[docs[0]])

BadRequestError: status_code: 400, body: {'message': 'invalid type: parameter texts is of type array but should be of type string. For proper usage, please refer to https://docs.cohere.com/embed-reference'}

In [18]:
import cohere

co = cohere.Client(COHERE_api_key)

response = co.embed(
    texts=["hello", "goodbye"], 
    # model="embed-english-v3.0",
    model="large",
    input_type="search_document"
)
print(response)

id='973c2f06-c691-4b37-9c83-346faef47e59' embeddings=[[1.6142578, 0.24841309, 0.5385742, -1.6630859, -0.27783203, 0.35888672, 1.3378906, -1.8261719, 0.89404297, 1.0791016, 1.0566406, 1.0664062, 0.20983887, -0.7558594, -0.29956055, -0.29248047, 0.23498535, -0.45947266, -0.7426758, -1.765625, 1.1699219, 2.0644531, 0.74902344, -1.6796875, 1.7685547, -0.7363281, 0.6376953, -1.6679688, -0.6621094, 1.2939453, 1.1904297, -2.3242188, 1.71875, 2.0117188, -0.46606445, 0.088134766, 1.6484375, -1.0292969, 0.07775879, -1.6503906, -0.035461426, 2.1660156, -1.2480469, -0.74072266, -0.53027344, -0.984375, -1.0039062, 0.47265625, -0.1574707, 1.984375, 0.72558594, 0.4885254, 0.25610352, -1.6113281, 1.7685547, -1.4824219, -3.4101562, -1.9023438, 0.47973633, -1.6972656, 0.34423828, 0.9121094, -0.046173096, 0.4482422, 0.828125, 1.7734375, -2.015625, -1.9423828, 0.63720703, -0.13000488, 2.5546875, -1.1972656, -1.0136719, -0.19714355, -0.43969727, 0.6591797, 0.46777344, -0.984375, -0.5629883, 0.014923096, 1.

In [37]:
[embeddings for embeddings in co.embed(
    texts=["hello", "goodbye"], 
    # model="embed-english-v3.0",
    model="large",
    input_type="search_document")
]

[('id', '92fa3642-4aab-4a2a-8199-7341408b3128'),
 ('embeddings',
  [[1.6142578,
    0.24841309,
    0.5385742,
    -1.6630859,
    -0.27783203,
    0.35888672,
    1.3378906,
    -1.8261719,
    0.89404297,
    1.0791016,
    1.0566406,
    1.0664062,
    0.20983887,
    -0.7558594,
    -0.29956055,
    -0.29248047,
    0.23498535,
    -0.45947266,
    -0.7426758,
    -1.765625,
    1.1699219,
    2.0644531,
    0.74902344,
    -1.6796875,
    1.7685547,
    -0.7363281,
    0.6376953,
    -1.6679688,
    -0.6621094,
    1.2939453,
    1.1904297,
    -2.3242188,
    1.71875,
    2.0117188,
    -0.46606445,
    0.088134766,
    1.6484375,
    -1.0292969,
    0.07775879,
    -1.6503906,
    -0.035461426,
    2.1660156,
    -1.2480469,
    -0.74072266,
    -0.53027344,
    -0.984375,
    -1.0039062,
    0.47265625,
    -0.1574707,
    1.984375,
    0.72558594,
    0.4885254,
    0.25610352,
    -1.6113281,
    1.7685547,
    -1.4824219,
    -3.4101562,
    -1.9023438,
    0.47973633,
    -

In [26]:
cohere_ef  = embedding_functions.CohereEmbeddingFunction(api_key=COHERE_api_key,  model_name="large")
cohere_ef(texts=["Hello, world!"])

TypeError: __call__() got an unexpected keyword argument 'texts'

In [42]:
doc.page_content

'Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.'

In [20]:
doc

Document(page_content='Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.')

In [245]:
# collection.add(ids=[str(uuid_name)], documents=response)

In [ ]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     # Set a really small chunk size, just to show.
#     chunk_size=100,
#     chunk_overlap=20,
#     length_function=len,
#     is_separator_regex=False,
# )

# texts = text_splitter.create_documents([state_of_the_union])
# print(texts[0])
# print(texts[1])

In [21]:
doc


Document(page_content='Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.')

In [24]:
type(doc.page_content)

str

In [27]:
co.embed(texts=["Hola"])

EmbedResponse_EmbeddingsFloats(id='8201de8a-8843-4ac2-9e94-82a7811852f3', embeddings=[[1.3359375, 0.52783203, -0.3540039, -0.10021973, 0.7055664, -0.64501953, -0.026916504, -1.9980469, -0.5361328, 0.30395508, 0.4580078, 0.018554688, -1.1572266, 0.07397461, -0.17163086, -1.1748047, 0.5834961, 2.2265625, 0.8378906, -1.4404297, 0.59765625, -0.39111328, 0.5234375, -0.21386719, 1.3183594, 4.328125, 1.4921875, -1.8759766, -2.2929688, 0.72558594, 1.3486328, -0.8417969, 0.5410156, 0.7636719, 1.1357422, -0.009765625, -0.1665039, 1.0507812, -1.9921875, 0.11462402, 1.0214844, 0.22070312, -1.8232422, -1.6386719, 0.03149414, -0.23242188, 0.7050781, 0.2944336, 0.19445801, 1.1230469, 0.8461914, -0.04711914, -2.6152344, -0.328125, -0.62597656, -2.7265625, -1.6074219, 0.16235352, -1.6367188, -0.4111328, 0.049804688, -0.010498047, 0.43896484, 0.35498047, 1.0361328, -0.14208984, 0.94677734, -2.5859375, -0.3881836, -1.3017578, 0.5336914, -1.1123047, -0.09667969, -1.7675781, -0.28222656, 0.6328125, 1.51562

In [32]:
doc.page_content.split(',')

['Ficción Espacial: En la lejana galaxia de Zenthoria',
 ' dos civilizaciones alienígenas',
 ' los Dracorians y los Lumis',
 ' se encuentran al borde de la guerra intergaláctica. Un intrépido explorador',
 ' Zara',
 ' descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos',
 ' Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.']

In [37]:
doc

Document(page_content='Características del Héroe Olvidado: Conocido como "Sombra Silenciosa", nuestro héroe es un maestro del sigilo y la astucia. Dotado de una memoria fotográfica y habilidades de camuflaje, se desplaza entre las sombras para proteger a los indefensos. Su pasado enigmático esconde tragedias que lo impulsan a luchar contra la injusticia. Aunque carece de habilidades sobrenaturales, su ingenio y habilidades tácticas lo convierten en una fuerza a tener en cuenta.')

crear base de datos

In [247]:
client.delete_collection(name="document_chunks")

In [24]:
client.list_collections()

[Collection(name=document_chunks)]

In [26]:
import uuid
import cohere
co = cohere.Client(COHERE_api_key)
collection_name = 'document_chunks'
# collection = client.get_or_create_collection(name=collection_name)
collection = client.get_or_create_collection(name=collection_name)
metadata_options = {
    "hnsw:space": "cosine"  # You can change this to "ip" or "cosine" if needed
}



In [ ]:
# for doc in docs:
#     uuid_name = uuid.uuid1()
#     embedding = co.embed(texts=doc.page_content.split(',')).embeddings[0]
#     print("document for", uuid_name)
#     # doc.page_content
    
#     collection.add(ids=[str(uuid_name)],
#                 embeddings=embedding,
#                 metadata=metadata_options,
#                 documents=doc.page_content)

embeding ok DB

In [244]:
list_parrafo

['Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.',
 'Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.',
 'Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como "Luz de Luna" florece solo dur

In [250]:
import numpy as np
for doc in list_parrafo:
    uuid_name = uuid.uuid1()
    embedding = co.embed(texts=[doc],model='embed-multilingual-v3.0',input_type ='search_query').embeddings[0] # search_query" or "search_document"
    print("document for", uuid_name)
    # doc.page_content
    
    collection.add(ids=[str(uuid_name)],
                embeddings=embedding,
                metadatas=metadata_options,
                documents=doc)

document for 6a47224f-349a-11ef-8452-4c77cb1e1a4e
document for 6a8489c1-349a-11ef-a7b4-4c77cb1e1a4e
document for 6ab8fbef-349a-11ef-9bc6-4c77cb1e1a4e
document for 6ae1f90e-349a-11ef-9120-4c77cb1e1a4e
document for 6b0d23cf-349a-11ef-b3ca-4c77cb1e1a4e


In [ ]:
Responde en el idioma de la ultima consulta realizada realizada.

In [125]:
import cohere

co = cohere.Client(COHERE_api_key)

        # Si la respuesta no esta en el contexto, Debes responder: "Lo siento, no tengo la información necesaria en este momento. ¿Hay algo más en lo que pueda ayudarte?cara triste"
        # Antes de responder por favor identifica el idioma del ultimo mensaje realizado y responde en el mismo idioma.
        # Después de cada respuesta por favor agrega emojis para que sea mas fácil de entender el texto de tu respuesta
        
response = co.chat(
    chat_history=[
        {"role": "SYSTEM", "message": f"""Debes ser lo más conciso y preciso posible en la entrega de información (20 palabras o menos)y traducir tu respuesta si es necesario a partir del siguiente contexto:
         {contexto}
        NOTA IMPORTANTE:

        "es": "Responde en una sola oración, en el mismo idioma que la pregunta, incluyendo emojis que resuman el contenido de la respuesta, y siempre en tercera persona.",
        "en": "Answer in one sentence, in the same language as the question, including emojis that summarize the content of the answer, and always in third person.",
        "pt": "Responda em uma frase, no mesmo idioma da pergunta, incluindo emojis que resumam o conteúdo da resposta, e sempre na terceira pessoa."
        """},
        
        {
            "role": "CHATBOT",
            "message": f"ya identifique el idioma del siguiente mensaje :{query} procedere a responder en el mismo idioma.",
        },
    ],
    message=f"""{query}""",
    # perform web search before answering the question. You can also use your own custom connector.
    connectors=[{"id": "web-search"}],
    seed=44,
    temperature=1,
    # model = 'command-r-plus',
    # model = 'command-r',
)

print(response.text)


Emma decidió compartir su día extra con el pueblo, dejando una huella imborrable en el corazón de sus habitantes. 💝🎢🌟


In [126]:
pip install langid

     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     - -------------------------------------- 0.1/1.9 MB 762.6 kB/s eta 0:00:03
     ---- ----------------------------------- 0.2/1.9 MB 1.5 MB/s eta 0:00:02
     -------- ------------------------------- 0.4/1.9 MB 2.0 MB/s eta 0:00:01
     ---------------- ----------------------- 0.8/1.9 MB 3.4 MB/s eta 0:00:01
     ---------------------------------- ----- 1.7/1.9 MB 5.9 MB/s eta 0:00:01
     ---------------------------------------- 1.9/1.9 MB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941180 sha256=920432967743e642a3211f53e70743272dc7feb4ea3199075e9d5df5bd9f2d42
  Stored in directory: c:\users\mavix\appdata\local\pip\cache\wheels\c5\01\a4\0160c55074707b535a6757a541842817d530d8080ca943a107
Succe

In [107]:
from langdetect import detect

language = detect(query)
print(language)

pt


In [128]:
import langid
langid.classify(query), query

(('en', -83.29462146759033), 'What did Emma decide to do?')

In [100]:
query

'What did Emma decide to do?'

In [69]:
Zara es un explorador intrépido que descubre un antiguo artefacto que podría contener la clave para la paz entre los Dracorians y los Lumis. 😮🛸🛡️


SyntaxError: invalid syntax (1135550763.py, line 1)

In [61]:
print(response.text)

¡Bienvenido a la galaxia de Zenthoria! 😁

En esta emocionante historia de ficción espacial, dos civilizaciones alienígenas, los Dracorians y los Lumis, están al borde de una devastadora guerra intergaláctica. 😱 Es aquí donde entra en escena Zara, un intrépido y valiente explorador. 🌟

Zara emprende un peligroso viaje a través de los confines de Zenthoria, explorando planetas hostiles y enfrentándose a desafíos cósmicos. 🌌 En su camino, se topa con un antiguo artefacto que se rumorea que posee el secreto para traer la paz a la galaxia. 🔑

A medida que la tensión aumenta y el destino de Zenthoria pende de un hilo, Zara debe descifrar los misterios encerrados en la reliquia. 💫 La carrera contra el tiempo es crucial, ya que cualquier retraso podría significar el estallido de la guerra y la destrucción de la galaxia. 🕰️

¿Podrá Zara descubrir a tiempo los secretos ocultos dentro del artefacto antiguo y traer la paz a la galaxia de Zentoria? 🤔 ¡El futuro de incontables vidas depende de su éx

In [77]:
contexto

'Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.'

In [ ]:
# Si la respuesta no esta en el contexto, Debes responder: "Lo siento, no tengo la información necesaria en este momento. ¿Hay algo más en lo que pueda ayudarte?cara triste"
# Antes de responder por favor identifica el idioma del ultimo mensaje realizado y responde en el mismo idioma.
# Después de cada respuesta por favor agrega emojis para que sea mas fácil de entender el texto de tu respuesta

In [ ]:
import cohere
import langid

In [ ]:
pip install --force-reinstall --no-deps matplotlib==3.8.2

In [272]:
language = langid.classify(query)[0]
if not language in ['es','en','pt']:
        response = co.chat(
        chat_history=[
            {"role": "SYSTEM", "message": f"""Eres  un detector muy preciso de idiomas,dime el idioma en una sola palabra (es, en, pt) de la siguiente pregunta : """},
        ],
        message=f"""{query}""",
        # connectors=[{"id": "translation"}],
        seed=44,
        temperature=0,
    )   

In [274]:
query

'Quem é Zara?'

In [273]:
response

NonStreamedChatResponse(text='pt', generation_id='a444794d-fc17-4ac2-801f-c9ced51c950d', citations=None, documents=None, is_search_required=None, search_queries=None, search_results=None, finish_reason='COMPLETE', tool_calls=None, chat_history=[Message_System(message='Eres  un detector muy preciso de idiomas,dime el idioma en una sola palabra (es, en, pt) de la siguiente pregunta : ', tool_calls=None, role='SYSTEM'), Message_User(message='Quem é Zara?', tool_calls=None, role='USER'), Message_Chatbot(message='pt', tool_calls=None, role='CHATBOT')], prompt=None, meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=35, output_tokens=1, search_units=None, classifications=None), tokens=ApiMetaTokens(input_tokens=105, output_tokens=1), warnings=None), response_id='11ed41a7-1cc8-4294-a412-1490444ccb8a')

In [131]:
@sleep_and_retry
@limits(calls=EMBED_LIMIT, period=60)
def generate_response(query:str, contexto:str):
    '''
    Description:
    ------------
    Esta función recibe un query y un contexto y devuelve una respuesta en el mismo idioma que el query.
    '''
    language = detect_language(query)

    dict_important1 =  {
            "es":"Debes ser lo más conciso y preciso posible en la entrega de información (20 palabras o menos) y traducir tu respuesta si es necesario a partir del siguiente contexto:",
            "en":"You must be as concise and precise as possible in delivering information (20 words or less) and translate your response if necessary from the following context:",
            "pt":"Você deve ser o mais conciso e preciso possível na entrega de informações (20 palavras ou menos) e traduzir sua resposta, se necessário, a partir do seguinte contexto:",
            }

    dict_important2 = {
            "es": "Responde en una sola oración, en el mismo idioma que la pregunta, incluyendo emojis que resuman el contenido de la respuesta, y siempre en tercera persona.",
            "en": "Answer in one sentence, in the same language as the question, including emojis that summarize the content of the answer, and always in third person.",
            "pt": "Responda em uma frase, no mesmo idioma da pergunta, incluindo emojis que resumam o conteúdo da resposta, e sempre na terceira pessoa.",
        }
    dict_important3  = {
            "es": "Responde en español, si el texto esta en español traduce la respuesta al español.",
            "en": "Answer in English, if the text is in español, translate the answer to English.",
            "pt": "Responda em português, se o texto estiver em español, traduza a resposta para o português.",
        }

    response = co.chat(
        chat_history=[
            {"role": "SYSTEM", "message": f"""{dict_important1[language]}
            {contexto}
            NOTA IMPORTANTE:
            {dict_important2[language]}

            """},
            
            # {
            #     "role": "CHATBOT",
            #     "message": f"ya identifique el idioma del siguiente mensaje :{query} procedere a responder en el mismo idioma.",
            # },
        ],
        message=f"""{query} {dict_important3[language]}""",
        #"realizar búsqueda web antes de responder a la pregunta. También puedes usar tu propio conector personalizado."
        # · contenedores de traducion y busqueda en contexto
        # connectors=[{"id": "translation"}],
        seed=44,
        temperature=0,
        # conversation_id='user_defined_id_1',
        # model = 'command-r-plus',
        # model = 'command-r',
    )
    respuesta =response.text

    return respuesta


def get_context(query:str,n_results:int=1):
    # query_embedding = co.embed(texts=[query]).embeddings[0]
    query_embedding = co.embed(texts=[query],model='embed-multilingual-v3.0',input_type ='search_query').embeddings[0] # search_query" or "search_document"
    results = collection.query(query_embeddings=query_embedding, n_results=n_results,include=['documents']) #include=['embeddings','documents']
    contexto = ' '.join([result for result in results['documents'][0]])
    # contexto  = results['documents'][0][0]
    contexto
    return results,contexto



In [ ]:
import cohere

co = cohere.Client(COHERE_api_key)

        # Si la respuesta no esta en el contexto, Debes responder: "Lo siento, no tengo la información necesaria en este momento. ¿Hay algo más en lo que pueda ayudarte?cara triste"
        # Antes de responder por favor identifica el idioma del ultimo mensaje realizado y responde en el mismo idioma.
        # Después de cada respuesta por favor agrega emojis para que sea mas fácil de entender el texto de tu respuesta
        
response = co.chat(
    chat_history=[
        {"role": "SYSTEM", "message": f"""Debes ser lo más conciso y preciso posible en la entrega de información (20 palabras o menos)y traducir tu respuesta si es necesario a partir del siguiente contexto:
         {contexto}
        NOTA IMPORTANTE:

        "es": "Responde en una sola oración, en el mismo idioma que la pregunta, incluyendo emojis que resuman el contenido de la respuesta, y siempre en tercera persona.",
        "en": "Answer in one sentence, in the same language as the question, including emojis that summarize the content of the answer, and always in third person.",
        "pt": "Responda em uma frase, no mesmo idioma da pergunta, incluindo emojis que resumam o conteúdo da resposta, e sempre na terceira pessoa."
        """},
        
        {
            "role": "CHATBOT",
            "message": f"ya identifique el idioma del siguiente mensaje :{query} procedere a responder en el mismo idioma.",
        },
    ],
    message=f"""{query}""",
    # perform web search before answering the question. You can also use your own custom connector.
    connectors=[{"id": "web-search"}],
    seed=44,
    temperature=1,
    # model = 'command-r-plus',
    # model = 'command-r',
)

print(response.text)


In [203]:
results

{'ids': [['4f560023-3450-11ef-b09a-4c77cb1e1a4e']],
 'distances': [[9138.305867231677]],
 'metadatas': [[{'hnsw:space': 'ip'}]],
 'embeddings': None,
 'documents': [['Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.']],
 'uris': None,
 'data': None}

In [78]:
# contexto  = context['documents'][0][0]
# contexto

'Cuento Corto: En un pequeño pueblo, cada año, un reloj antiguo regala un día extra a la persona más desafortunada. Emma, una joven huérfana, es la elegida este año. Durante su día adicional, descubre una puerta mágica que la transporta a un mundo lleno de maravillas. Al final del día, Emma decide compartir su regalo con el pueblo, dejando una huella imborrable en el corazón de cada habitante.'

In [204]:
generate_response(query, contexto)

KeyError: 'ca'

In [299]:

# query = "Quien es Zara?"
# # query = 'What did Emma decide to do?'
# # query = 'What is the name of the magical flower?'
# context = get_context(query)
# # response = generate_response(query, context)
# # print(response)


In [139]:
langid.classify(query)[0]

'es'

In [13]:
import langid
langid.classify(query), query

NameError: name 'query' is not defined

In [211]:
results

{'ids': [['288fc59c-3498-11ef-95fa-4c77cb1e1a4e']],
 'distances': [[1.2475253573695065e-05]],
 'metadatas': [[{'hnsw:space': 'ip'}]],
 'embeddings': None,
 'documents': [['']],
 'uris': None,
 'data': None}

TEST PROMPT

In [ ]:
PRUEBAS 
Se debera responder correctamente a las preguntas que se hagan sobre los documentos, algunos 
ejemplos son: 
- Quien es Zara? 
- What did Emma decide to do? 
- What is the name of the magical flower?

In [ ]:
pruebas de deteccion de idioma
Query 1
Inglés: Who is Zara?
Español: ¿Quién es Zara?
Portugués: Quem é Zara?
Query 2
Inglés: What did Emma decide to do?
Español: ¿Qué decidió hacer Emma?
Portugués: O que Emma decidiu fazer?
Query 3
Inglés: What is the name of the magical flower?
Español: ¿Cuál es el nombre de la flor mágica?
Portugués: Qual é o nome da flor mágica?
¿Hay algo más en lo que pueda ayudarte?

In [44]:
@sleep_and_retry
@limits(calls=EMBED_LIMIT, period=60)
def detect_language(query:str):
    '''
    Description:
    ------------
    Esta función recibe un query y devuelve el idioma del query.
    '''
    language = langid.classify(query)[0]
    if not language in ['es','en','pt']:
        response = co.chat(
        chat_history=[
            {"role": "SYSTEM", "message": f"""Eres  un detector muy preciso de idiomas,dime el idioma en una sola palabra (es, en, pt) de la siguiente pregunta : """},
        ],
        message=f"""{query}""",
        # connectors=[{"id": "translation"}],
        seed=44,
        temperature=0,
        )
        language = response.text
    return language

In [152]:

# query ='quien es Zara?'
# query ='Who is Zara?'
# query ='Quem é Zara?'
# query ='What did Emma decide to do?'
# query ='¿Qué decidió hacer Emma?'
# query ='O que Emma decidiu fazer?'
query ='What is the name of the magical flower?'
# query ='¿Cuál es el nombre de la flor mágica?'
# query ='Qual é o nome da flor mágica?'
# query ='de que trata el texto?'

results,contexto = get_context(query,5)
respuesta = generate_response(query,contexto)
print(respuesta)



The magical flower is called the "Moonlight" 🌺.


In [129]:
contexto = ' '.join([result for result in results['documents'][0]])

In [130]:
contexto

'Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia. Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos. Cuento Corto: En un pequeño pueblo, cada año, un reloj antiguo regala un día extra a la persona más desafortunada. Emma, una joven h

In [128]:
for result in results['documents'][0]:
    print(result)
    print('')

Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.

Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.

Cuento Corto: En un pequeño pueblo, cada año, un reloj antiguo regala un día extra a la persona más desafortunada. Emma, una joven 

In [124]:
results['documents']

[['Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.',
  'Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.',
  'Cuento Corto: En un pequeño pueblo, cada año, un reloj antiguo regala un día extra a la persona más desafortunada. Emma,

In [116]:
results

{'ids': [['6a8489c1-349a-11ef-a7b4-4c77cb1e1a4e',
   '6a47224f-349a-11ef-8452-4c77cb1e1a4e',
   '6ae1f90e-349a-11ef-9120-4c77cb1e1a4e',
   '6ab8fbef-349a-11ef-9bc6-4c77cb1e1a4e',
   '6b0d23cf-349a-11ef-b3ca-4c77cb1e1a4e']],
 'distances': None,
 'metadatas': None,
 'embeddings': None,
 'documents': [['Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia.',
   'Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la

In [ ]:
Emma escolheu compartilhar sua aventura mágica com os habitantes da cidade, deixando uma lembrança inesquecível para todos. 😊🤲🌟


In [ ]:
test_respuesta

In [ ]:
Zara é um intrépido explorador em uma missão para desvendar os mistérios de um antigo artefato alienígena e trazer paz à galáxia de Zenthoria. 🚀🛸🌌️


In [59]:
respuesta[0]

'Zara is an intrepid explorer on a mission to unravel the mysteries of an ancient artifact that holds the key to intergalactic peace. 🚀'

In [ ]:
('Zara es un valiente explorador que descubre un antiguo artefacto que podría ser clave para evitar una guerra intergaláctica. 🌌️🔑🛡️',)
('Zara es un valiente explorador que descubre un antiguo artefacto que podría ser clave para evitar una guerra intergaláctica. 🌌️🔑🛡️',)
('Zara es un valiente explorador que descubre un antiguo artefacto que podría ser clave para evitar una guerra intergaláctica. 🌌️🔑🛡️',)

In [ ]:
('Zara is an intrepid explorer on a mission to unravel the mysteries of an ancient artifact that holds the key to intergalactic peace. 🚀',)
('Zara is an intrepid explorer on a mission to unravel the mysteries of an ancient artifact that holds the key to intergalactic peace. 🚀',)
('Zara is an intrepid explorer on a mission to unravel the mysteries of an ancient artifact that holds the key to intergalactic peace. 🚀',)

In [43]:
import time
from ratelimit import limits, sleep_and_retry

# Limites del API de Cohere
EMBED_LIMIT = 5  # llamadas por minuto
RERANK_CHAT_LIMIT = 10  # llamadas por minuto
OTHER_LIMIT = 100  # llamadas por minuto

# @sleep_and_retry
# @limits(calls=EMBED_LIMIT, period=60)

In [ ]:
# pip install ratelimit

In [ ]:
query ='Who is Zara?'
query ='What did Emma decide to do?'
results,contexto = get_context(query)
generate_response(query,contexto)


('Emma chose to share her magical day with the town, leaving a lasting impression on its people. 🎁🤍🌟',)
('Emma chose to share her magical day with the town, leaving a lasting impression on its people. 🎁🤍🌟',)
('Emma chose to spread joy, opening a magical portal for others to explore, leaving the town with unforgettable memories. 🎁🤍🌌',)
('Emma chose to spread joy, opening a magical portal for others to explore, leaving the town with unforgettable memories. 🎁🤍🌌',)
('Emma chose to spread joy, opening a magical portal for others to explore, leaving the town with unforgettable memories. 🎁🤍🌌',)
('Emma chose to spread joy, opening a magical portal for others to explore, leaving the town with unforgettable memories. 🎁🤍🌌',)
('Emma chose to spread joy, opening a magical portal for others to explore, leaving the town with unforgettable memories. 🎁🤍🌌',)





In [ ]:
¿Qué decidió hacer Emma?

In [316]:
query ='Who is Zara?'
results,contexto = get_context(query)
generate_response(query,contexto)


'Pruebas pasadas con éxito'

In [283]:
query ='Who is Zara?'
query ='quien es el presidente de los estados unido?'
response = co.chat(
chat_history=[
    {"role": "SYSTEM", "message": f"""Eres  un detector muy preciso de idiomas,dime el idioma en una sola palabra (es, en, pt) de la siguiente pregunta : """},
],
message=f"""{query}""",
# connectors=[{"id": "translation"}],
seed=44,
temperature=0,
)

response

NonStreamedChatResponse(text='es', generation_id='df6a5a95-3c39-465d-b679-61df142d7f96', citations=None, documents=None, is_search_required=None, search_queries=None, search_results=None, finish_reason='COMPLETE', tool_calls=None, chat_history=[Message_System(message='Eres  un detector muy preciso de idiomas,dime el idioma en una sola palabra (es, en, pt) de la siguiente pregunta : ', tool_calls=None, role='SYSTEM'), Message_User(message='quien es el presidente de los estados unido?', tool_calls=None, role='USER'), Message_Chatbot(message='es', tool_calls=None, role='CHATBOT')], prompt=None, meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=40, output_tokens=1, search_units=None, classifications=None), tokens=ApiMetaTokens(input_tokens=109, output_tokens=1), warnings=None), response_id='eef210ed-78e6-4e92-bb6a-4578e1447079')

In [258]:
contexto

'Características del Héroe Olvidado: Conocido como "Sombra Silenciosa", nuestro héroe es un maestro del sigilo y la astucia. Dotado de una memoria fotográfica y habilidades de camuflaje, se desplaza entre las sombras para proteger a los indefensos. Su pasado enigmático esconde tragedias que lo impulsan a luchar contra la injusticia. Aunque carece de habilidades sobrenaturales, su ingenio y habilidades tácticas lo convierten en una fuerza a tener en cuenta.'

In [300]:
# results['embeddings'][0][0]

In [182]:
language = langid.classify(query)[0]
language

'fr'

In [184]:
query ='Who is Zara?'

In [189]:
query = "O que Emma decidiu fazer?"

In [190]:
langid.classify(query)

('ca', -61.328288555145264)

In [172]:
contexto

'Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos.'

In [137]:
# test_text = "Este es un texto de prueba."
# embeddings = cohere_ef([test_text])

In [138]:
# cohere_ef.embed_query(input = ["Hola mundo"],retry_kwargs={"max_retries": 3, "wait_time": 1})

In [42]:
test_text = "Este es un texto de prueba."
embeddings = cohere_ef([test_text])

# Asegúrate de que el embedding sea una lista de listas
if not isinstance(embeddings, list) or not all(isinstance(e, list) for e in embeddings):
    embeddings = [list(e) for e in embeddings]

print(embeddings)


ValueError: Expected each embedding in the embeddings to be a list, got ['tuple']

In [39]:
client.list_collections()

In [ ]:
text_spliter_chunck()

In [19]:
ntoken_list

[134, 104, 118, 112, 127]

In [12]:
[paragraph.text for paragraph in doc.paragraphs]


NameError: name 'doc' is not defined

In [11]:
doc.paragraphs

NameError: name 'doc' is not defined

In [16]:
def text_to_chromadb(pdf):

    temp_dir = tempfile.TemporaryDirectory()
    temp_filepath = os.path.join(temp_dir.name, pdf.name)
    with open(temp_filepath, "wb") as f:
        f.write(pdf.getvalue())



    loader = PyPDFLoader(temp_filepath)
    text = loader.load()



    with st.spinner(f'Creando embedding fichero: {pdf.name}'):
        create_embeddings(pdf.name, text)




    return True

In [ ]:
import os
import streamlit as st



import chromadb
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings





FILE_LIST = "archivos.txt"
INDEX_NAME = 'taller'



chroma_client = chromadb.HttpClient(host='localhost', port=8000)



def save_name_files(path, new_files):



    old_files = load_name_files(path)



    with open(path, "a") as file:
        for item in new_files:
            if item not in old_files:
                file.write(item + "\n")
                old_files.append(item)
    
    return old_files





def load_name_files(path):



    archivos = []
    with open(path, "r") as file:
        for line in file:
            archivos.append(line.strip())



    return archivos





def clean_files(path):
    with open(path, "w") as file:
        pass
    chroma_client.delete_collection(name=INDEX_NAME)
    collection = chroma_client.create_collection(name=INDEX_NAME)



    return True





def text_to_chromadb(pdf):



    temp_dir = tempfile.TemporaryDirectory()
    temp_filepath = os.path.join(temp_dir.name, pdf.name)
    with open(temp_filepath, "wb") as f:
        f.write(pdf.getvalue())



    loader = PyPDFLoader(temp_filepath)
    text = loader.load()



    with st.spinner(f'Creando embedding fichero: {pdf.name}'):
        create_embeddings(pdf.name, text)



    return True





def create_embeddings(file_name, text):
    print(f"Creando embeddings del archivo: {file_name}")



    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        length_function=len
        )        
    
    chunks = text_splitter.split_documents(text)



    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        )
    
    Chroma.from_documents(
        chunks,
        embeddings,   
        client=chroma_client,
        collection_name=INDEX_NAME)
        
    return True

In [ ]:
def print_collections(collections:list) -> None:

    for collection in collections:
        print(f"Collection(name={collection.name})")

def print_verbose(msg: str,verbose:bool) -> None:
    print(msg) if verbose else None



def num_tokens_from_string(string: str, encoding_name: str="cl100k_base") -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def doc_to_paragraphs(file:str='F:\Proyectos\PiConsulting\challenge_rag_llm\docs\documento.docx'):
    # Abre el archivo .docx
    doc = Document(file)

    list_parrafo = [parrafo.text for parrafo in doc.paragraphs if (parrafo.text != "")and(parrafo.text != " ")]
    ntoken_list = [num_tokens_from_string(parrafo) for parrafo in list_parrafo]

    return ntoken_list, list_parrafo



    
def split_text(
        input: Union[str,
        List[str]],
        unir_list:bool=False, 
        chunk_size: int=300,
        chunk_overlap: int=0,
        separators:list=["\n\n", "\n",],
        verbose=False
)->list:

    if isinstance(input, str):
        input = [input]
    elif isinstance(input, list):
        if unir_list:
            input = [" ".join(input)]
    print_verbose(input,verbose)

    # Configuración del splitter
    text_splitter = RecursiveCharacterTextSplitter(
        separators=separators,          # Separadores a utilizar
        chunk_size=chunk_size,          # Tamaño de cada chunk
        chunk_overlap=chunk_overlap,    # Solapamiento entre chunks
    )

    # Crear documentos divididos
    docs = text_splitter.create_documents(input)

    # Mostrar resultado
    for doc in docs:
        print_verbose(msg=f'Chunk: {doc.page_content}', verbose=verbose)

    return docs


def get_context(collection:Collection,query:str,n_results:int=1):
    # query_embedding = co.embed(texts=[query]).embeddings[0]
    query_embedding = co.embed(texts=[query],model='embed-multilingual-v3.0',input_type ='search_query').embeddings[0] # search_query" or "search_document"
    results = collection.query(query_embeddings=query_embedding, n_results=n_results,include=['documents']) #include=['embeddings','documents']
    contexto = ' '.join([result for result in results['documents'][0]])
    # contexto  = results['documents'][0][0]
    contexto
    return results,contexto

def add_documents_to_collection(
        collection:Collection,
        docs:list,
        model:str ='embed-multilingual-v3.0',
        input_type: Optional[EmbedInputType]='search_query',
        embedding_types:Optional[EmbedInputType]= None,
        metadata_options:dict = dict(),

    )->Collection:
    for doc in docs:
        doc = doc.page_content
        uuid_name = uuid.uuid1()
        embedding = co.embed(texts=[doc], model=model, input_type=input_type,
            embedding_types=embedding_types).embeddings[0] # search_query" or "search_document"
        print("document for", uuid_name)
        # doc.page_content
        
        collection.add(ids=[str(uuid_name)],
                    embeddings=embedding,
                    metadatas=metadata_options,
                    documents=doc)
    return collection

@sleep_and_retry
@limits(calls=EMBED_LIMIT, period=60)
def detect_language(query:str):
    language = langid.classify(query)[0]
    if not language in ['es','en','pt']:
        response = co.chat(
        chat_history=[
            {"role": "SYSTEM", "message": f"""Eres  un detector muy preciso de idiomas,dime el idioma en una sola palabra (es, en, pt) de la siguiente pregunta : """},
        ],
        message=f"""{query}""",
        # connectors=[{"id": "translation"}],
        seed=44,
        temperature=0,
        )
        language = response.text
    return language

@sleep_and_retry
@limits(calls=EMBED_LIMIT, period=60)
def generate_response(query:str, contexto:str):

    language = detect_language(query)

    dict_important1 =  {
            "es":"Debes ser lo más conciso y preciso posible en la entrega de información (20 palabras o menos) y traducir tu respuesta si es necesario a partir del siguiente contexto:",
            "en":"You must be as concise and precise as possible in delivering information (20 words or less) and translate your response if necessary from the following context:",
            "pt":"Você deve ser o mais conciso e preciso possível na entrega de informações (20 palavras ou menos) e traduzir sua resposta, se necessário, a partir do seguinte contexto:",
            }

    dict_important2 = {
            "es": "Responde en una sola oración, en el mismo idioma que la pregunta, incluyendo emojis que resuman el contenido de la respuesta, y siempre en tercera persona.",
            "en": "Answer in one sentence, in the same language as the question, including emojis that summarize the content of the answer, and always in third person.",
            "pt": "Responda em uma frase, no mesmo idioma da pergunta, incluindo emojis que resumam o conteúdo da resposta, e sempre na terceira pessoa.",
        }
    dict_important3  = {
            "es": "Responde en español, si el texto esta en español traduce la respuesta al español.",
            "en": "Answer in English, if the text is in español, translate the answer to English.",
            "pt": "Responda em português, se o texto estiver em español, traduza a resposta para o português.",
        }

    response = co.chat(
        chat_history=[
            {"role": "SYSTEM", "message": f"""{dict_important1[language]}
            {contexto}
            NOTA IMPORTANTE:
            {dict_important2[language]}

            """},
            
            # {
            #     "role": "CHATBOT",
            #     "message": f"ya identifique el idioma del siguiente mensaje :{query} procedere a responder en el mismo idioma.",
            # },
        ],
        message=f"""{query} {dict_important3[language]}""",
        #"realizar búsqueda web antes de responder a la pregunta. También puedes usar tu propio conector personalizado."
        # · contenedores de traducion y busqueda en contexto
        # connectors=[{"id": "translation"}],
        seed=44,
        temperature=0,
        # conversation_id='user_defined_id_1',
        # model = 'command-r-plus',
        # model = 'command-r',
    )
    respuesta =response.text

    return respuesta


# Flujos de trabajo 
import os
from dotenv import load_dotenv

import chromadb

from docx import Document
import tiktoken
from typing import Union, List

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langdetect import detect
import langid
import cohere

import uuid
import cohere

import time
from ratelimit import limits, sleep_and_retry

from typing import Sequence, Optional, Union,Literal,Any
from chromadb.api.models.Collection import Collection
EmbedRequestTruncate = Union[Literal["NONE", "START", "END"], Any]
EmbeddingType = Union[Literal["float", "int8", "uint8", "binary", "ubinary"], Any]
EmbedInputType = Union[ Literal["search_document", "search_query", "classification", "clustering"], Any]


import numpy as np
# Limites del API de Cohere
EMBED_LIMIT = 5  # llamadas por minuto
RERANK_CHAT_LIMIT = 10  # llamadas por minuto
OTHER_LIMIT = 100  # llamadas por minuto


load_dotenv()
COHERE_api_key: str = os.getenv("COHERE_API_KEY")

co = cohere.Client(COHERE_api_key)


ntoken_list, list_parrafo = doc_to_paragraphs('F:\Proyectos\PiConsulting\challenge_rag_llm\docs\documento.docx')
docs = split_text(input=list_parrafo, chunk_size=500, chunk_overlap=20,unir_list=False,separators=None,verbose=False)#separators=["\n\n", "\n"]


client = chromadb.PersistentClient(path="./../app/db/ChromaDB/")
collection = client.get_or_create_collection(name="document_chunks")
model = 'embed-multilingual-v3.0'
input_type = 'search_query'
embedding_types = None
metadata_options = {
    "hnsw:space": "cosine"  # You can change this to "ip" or "cosine" if needed
}
collection = add_documents_to_collection(
    collection,
    docs,
    model= model,
    input_type=input_type,
    metadata_options=metadata_options,
    embedding_types=embedding_types)

# query ='quien es Zara?'
# query ='Who is Zara?'
# query ='Quem é Zara?'
# query ='What did Emma decide to do?'
# query ='¿Qué decidió hacer Emma?'
# query ='O que Emma decidiu fazer?'
query ='What is the name of the magical flower?'
# query ='¿Cuál es el nombre de la flor mágica?'
# query ='Qual é o nome da flor mágica?'
# query ='de que trata el texto?'

results,contexto = get_context(collection,query,5)
respuesta = generate_response(query,contexto)
print(respuesta)

test_language_data = {'quien es Zara?':'es',
                        'Who is Zara?':'en',
                        'Quem é Zara?':'pt',
                        'What did Emma decide to do?':'en',
                        '¿Qué decidió hacer Emma?':'es',
                        'O que Emma decidiu fazer?':'pt',
                        'What is the name of the magical flower?':'en',
                        '¿Cuál es el nombre de la flor mágica?':'es',
                        'Qual é o nome da flor mágica?':'pt'}

def test_detect_language(test_language_data):
    '''
    Función que realiza pruebas de detección de idioma
    Description: 
    ------------
    Esta función recibe un diccionario con preguntas y su respectivo idioma y devuelve un mensaje de éxito si las pruebas pasan.
    la variable test_language_data es un diccionario con preguntas y su respectivo idioma se encuentra en el config_test.yaml.   
 
    '''
    for query,language in test_language_data.items():
        assert detect_language(query) == language # Que hace assert: 
    return 'Test Detectar lenguaje: [PASS]'

test_detect_language(test_language_data)




from pydantic import BaseModel

class UserRequest(BaseModel):
    user_name: str
    question: str

In [ ]:
from pydantic import BaseModel

class UserRequest(BaseModel):
    user_name: str
    question: str

ValueError: Expected each embedding in the embeddings to be a list, got ['queien es Zara']

In [36]:
pwd

'f:\\Proyectos\\PiConsulting\\challenge_rag_llm\\Notebooks'

In [39]:
import os
os.chdir('F:\\Proyectos\\PiConsulting\\challenge_rag_llm')
from app.utils.funcions_general import ConfigManager
config = ConfigManager(config_paths=['./config/test/unit.yaml'])
request = config.get(['test_get_response'])

In [51]:
(request['user_name'],request['question'])

('mvx', 'Quien es Zara')

In [42]:
request['question']

'Quien es Zara'